# GDP


In [ ]:
import requests
import pandas as pd
from io import BytesIO
import gzip

url = "https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nama_10_gdp.tsv.gz"

# Send a GET request to the URL to download the file
response = requests.get(url)

if response.status_code == 200:
    # Extract the content of the response and load it as a DataFrame
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as f:
        df_gdp = pd.read_csv(f, sep='\t')

    # Now you have the data in a Pandas DataFrame (df)
    # You can work with the DataFrame as needed

else:
    print("Failed to download the file")

In [ ]:
geocode_mapping = {
    "AL": "Albania",
    "AT": "Austria",
    "BA": "Bosnia and Herzegovia",
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CH": "Switzerland",
    "CY": "Cyprus",
    "CZ": "Czechia",
    "DE": "Germany",
    "DK": "Denmark",
    "EA": "Euro Area",
    "EA12": "Euro Area - 2001-2006",
    "EU15": "Euro Area - 1995-2004",
    "EA19": "Euro Area - 2019",
    "EA20": "Euro Area - 2020",
    "EE": "Estonia",
    "EL": "Greece",
    "ES": "Spain",
    "EU": "European Union",
    "EU27_2020":"European Union 27",
    "EU28": "European Union 28",
    "FI": "Finland",
    "FR": "France",
    "HR": "Croatia",
    "HU": "Hungary",
    "IE": "Ireland",
    "IS": "Iceland",
    "IT" :"Italy",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "LV": "Latvia",
    "ME": "Montenegro",
    "MK": "North Macedonia",
    "MT": "Malta",
    "NL": "Netherlands",
    "NO": "Norway",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "RS": "Serbia",
    "SE": "Sweden",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "TR": "Turkey",
    "UK": "United Kingdom",
    "US": "United States",
    "XK" : "Kosovo"
}

In [ ]:
# Split the 'column_to_split' into separate columns
# Use double backslashes ('\\') in 'geo\\time' to escape the backslash
df_gdp[['unit', 'na_item', 'geo\\time']] = df_gdp['unit,na_item,geo\\time'].str.split(',', expand=True)

df_gdp=df_gdp[df_gdp["unit"] == "CLV05_MEUR"]


In [ ]:
# Map geo\\time to geocode_mapping
df_gdp.loc[:, "Country"] = df_gdp['geo\\time'].map(geocode_mapping)

# Drop the original column if needed
df_gdp.drop(columns=['unit,na_item,geo\\time', 'unit', 'na_item', 'geo\\time' ], inplace=True)

# Drop rows with NaN index names
df_gdp.dropna(subset=['Country'], inplace=True)

df_gdp.set_index(["Country"], inplace=True)

<ipython-input-114-02dec04e062b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-114-02dec04e062b>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-114-02dec04e062b>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_gdp.columns

Index(['2022 ', '2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ',
       '2014 ', '2013 ', '2012 ', '2011 ', '2010 ', '2009 ', '2008 ', '2007 ',
       '2006 ', '2005 ', '2004 ', '2003 ', '2002 ', '2001 ', '2000 ', '1999 ',
       '1998 ', '1997 ', '1996 ', '1995 ', '1994 ', '1993 ', '1992 ', '1991 ',
       '1990 ', '1989 ', '1988 ', '1987 ', '1986 ', '1985 ', '1984 ', '1983 ',
       '1982 ', '1981 ', '1980 ', '1979 ', '1978 ', '1977 ', '1976 ', '1975 '],
      dtype='object')

In [ ]:
import re
import numpy as np
# Function to clean and convert a column
def clean_and_convert_column(column):
    def clean_and_convert(x):
        try:
            return float(re.sub(r'[^0-9.]', '', str(x)))
        except ValueError:
            return np.nan
    return column.apply(clean_and_convert)

# Apply the function to all columns (excluding the index columns)
for column in df_gdp.columns:
    if column not in ['unit,na_item,geo\\time']:
        df_gdp[column] = clean_and_convert_column(df_gdp[column])
df_gdp = df_gdp[~(df_gdp.iloc[:, 1:] < 100).all(axis=1)]


<ipython-input-116-798bdf974231>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_gdp = df_gdp.transpose()
df_gdp = df_gdp.iloc[:, ::-1]

In [ ]:
df_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 2022  to 1975 
Columns: 954 entries, United Kingdom to Austria
dtypes: float64(954)
memory usage: 359.2+ KB


In [ ]:
df_gdp.index

Index(['2022 ', '2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ',
       '2014 ', '2013 ', '2012 ', '2011 ', '2010 ', '2009 ', '2008 ', '2007 ',
       '2006 ', '2005 ', '2004 ', '2003 ', '2002 ', '2001 ', '2000 ', '1999 ',
       '1998 ', '1997 ', '1996 ', '1995 ', '1994 ', '1993 ', '1992 ', '1991 ',
       '1990 ', '1989 ', '1988 ', '1987 ', '1986 ', '1985 ', '1984 ', '1983 ',
       '1982 ', '1981 ', '1980 ', '1979 ', '1978 ', '1977 ', '1976 ', '1975 '],
      dtype='object')

In [ ]:
import plotly.express as px
# Reshape the DataFrame
df_gdp_reshaped = df_gdp.stack().reset_index()
df_gdp_reshaped.columns = ['Year', 'Country', 'GDP']
df_grouped = df_gdp_reshaped.groupby(['Country', 'Year'])['GDP'].mean().unstack("Country")

In [ ]:

# Assuming df_grouped is your grouped DataFrame
# If not, replace it with the actual name of your DataFrame

# Reset the index to make 'Country' and 'Year' regular columns
df_grouped_reset = df_grouped.reset_index()

# Melt the DataFrame to have a single 'Country' column
df_melted = pd.melt(df_grouped_reset, id_vars=['Year'], value_vars=df_grouped_reset.columns[1:],
                    var_name='Country', value_name='GDP')

# Create the line chart using Plotly Express
fig = px.line(df_melted, x='Year', y='GDP', color='Country',
              title='GDP Over Time by Country',
              template='plotly_dark')

# fig.update_layout(width=800, height=400)
fig.show()


In [ ]:
df_melted

,Year,Country,GDP
0,1975,Albania,NaN
1,1976,Albania,NaN
2,1977,Albania,NaN
3,1978,Albania,NaN
4,1979,Albania,NaN
...,...,...,...
2059,2018,United Kingdom,974990.469565
2060,2019,United Kingdom,992090.547826
2061,2020,United Kingdom,NaN
2062,2021,United Kingdom,NaN


In [ ]:
# List of columns to keep (exclude "European Union")
columns_to_keep = [col for col in df_gdp.columns if "Euro" not in col]

# Filter columns based on the list
df_gdp = df_gdp[columns_to_keep]

# Calculate mean, median, mode, min, and max for each column
mean_values = df_gdp.mean()
median_values = df_gdp.median()
mode_values = df_gdp.mode().iloc[0]  # Handling multi-modal distributions
min_values = df_gdp.min()
max_values = df_gdp.max()

# Create a new DataFrame with the summary statistics
df_gdp_summary = pd.DataFrame({
    'Mean': mean_values,
    'Median': median_values,
    'Mode': mode_values,
    'Min': min_values,
    'Max': max_values
})

# Display or use df_gdp_summary as needed
df_gdp_summary=df_gdp_summary.groupby("Country")["Mean", "Median", "Mode", "Min", "Max"].sum()

<ipython-input-123-6a2c60bc7f6e>:24: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
# Find the index label where the minimum value in the "Min" column is located
min_row_index = df_gdp_summary["Min"].idxmin()

# Extract the entire row based on the index label
min_row = df_gdp_summary.loc[min_row_index]

min_row

Mean      1.321251e+06
Median    1.469128e+06
Mode      6.072324e+05
Min       6.072324e+05
Max       2.009803e+06
Name: Albania, dtype: float64

In [ ]:
# Find the index label where the maximum value in the "Max" column is located
max_row_index = df_gdp_summary["Max"].idxmax()

# Extract the entire row based on the index label
max_row = df_gdp_summary.loc[max_row_index]

# Display the result
print(max_row)

Mean      5.045670e+08
Median    4.994288e+08
Mode      3.774104e+08
Min       3.749926e+08
Max       6.513644e+08
Name: Germany, dtype: float64


In [ ]:
print("min row", min_row, "max value", max_row)

min row Mean      1.321251e+06
Median    1.469128e+06
Mode      6.072324e+05
Min       6.072324e+05
Max       2.009803e+06
Name: Albania, dtype: float64 max value Mean      5.045670e+08
Median    4.994288e+08
Mode      3.774104e+08
Min       3.749926e+08
Max       6.513644e+08
Name: Germany, dtype: float64


# Immigration by age group, sex and level of human development of the country of citizenship

In [ ]:
import requests
import pandas as pd
import gzip
from io import BytesIO

# The URL of the TSV file
tsv_url = "https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/migr_imm9ctz.tsv.gz"

# Download the TSV file
response = requests.get(tsv_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the response and load it as a DataFrame
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as f:
        df_mig = pd.read_csv(f, sep='\t')

    # Now you have the data in a Pandas DataFrame (df_pop)
    # You can work with the DataFrame as needed

else:
    print("Failed to download the file")


In [ ]:
df_mig

,"citizen,agedef,age,unit,sex,geo\time",2021,2020,2019,2018,2017,2016,2015,2014,2013
0,"CC5_13_FOR_X_IS,COMPLET,TOTAL,NR,F,AT",:,:,3557,3453,:,4242,4203,3765,3846
1,"CC5_13_FOR_X_IS,COMPLET,TOTAL,NR,F,BE",:,:,1638,1504,:,1171,:,1175,1640
2,"CC5_13_FOR_X_IS,COMPLET,TOTAL,NR,F,BG",:,:,2126 p,1871,:,1201,925,869,993 p
3,"CC5_13_FOR_X_IS,COMPLET,TOTAL,NR,F,CH",:,:,2382,2488,:,2342,2264,2213,2248
4,"CC5_13_FOR_X_IS,COMPLET,TOTAL,NR,F,CY",:,:,:,:,:,:,:,:,24
...,...,...,...,...,...,...,...,...,...,...
66997,"UNK,REACH,Y_LT5,NR,T,NO",9,0,0,2,5,14,11,11,18
66998,"UNK,REACH,Y_LT5,NR,T,PL",:,1 ep,0 ep,0 ep,0 ep,0 ep,2 p,0 p,0
66999,"UNK,REACH,Y_LT5,NR,T,PT",0,0,0,0,0 e,0 e,0 e,0,0
67000,"UNK,REACH,Y_LT5,NR,T,SE",57,68,122,114,158,109,65,61,41


In [ ]:
import re
import numpy as np
# Function to clean and convert a column
def clean_and_convert_column(column):
    def clean_and_convert(x):
        try:
            return float(re.sub(r'[^0-9.]', '', str(x)))
        except ValueError:
            return np.nan
    return column.apply(clean_and_convert)

# Apply the function to all columns (excluding the index columns)
for column in df_mig.columns:
    if column not in ['citizen,agedef,age,unit,sex,geo\\time']:
        df_mig[column] = clean_and_convert_column(df_mig[column])

df_mig.fillna(method='ffill', inplace=True)

df_mig = df_mig[~(df_mig.iloc[:, 1:] < 100).all(axis=1)]


In [ ]:
# Split the 'column_to_split' into separate columns
# Use double backslashes ('\\') in 'geo\\time' to escape the backslash
# Split the 'citizen,agedef,age,unit,sex,geo\time' column
df_mig[['citizen', 'agedef', 'age', 'unit', 'sex', 'country']] = df_mig['citizen,agedef,age,unit,sex,geo\\time'].str.split(',', expand=True)

# Now, use .loc to assign the values to the desired columns
df_mig.loc[:, 'citizen'] = df_mig['citizen'].astype(str)
df_mig.loc[:, 'agedef'] = df_mig['agedef'].astype(str)
df_mig.loc[:, 'age'] = df_mig['age'].astype(str)
df_mig.loc[:, 'unit'] = df_mig['unit'].astype(str)
df_mig.loc[:, 'sex'] = df_mig['sex'].astype(str)
df_mig.loc[:, 'country'] = df_mig['country'].astype(str)

<ipython-input-130-be53f35caf59>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-130-be53f35caf59>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-130-be53f35caf59>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-130-be5

In [ ]:
df_mig=df_mig[df_mig["citizen"] == "CC5_13_FOR_X_IS"]

In [ ]:
# Map geo\\time to geocode_mapping
df_mig.loc[:, "country"] = df_mig['country'].map(geocode_mapping)

# Drop the original column if needed
# df_gdp.drop(columns=['unit,na_item,geo\\time', 'unit', 'na_item', 'geo\\time' ], inplace=True)

# Drop rows with NaN index names
df_mig.dropna(subset=['country'], inplace=True)

df_mig.set_index(["country"], inplace=True)

<ipython-input-132-56dd200e967b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-132-56dd200e967b>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_mig.index

Index(['Austria', 'Belgium', 'Bulgaria', 'Switzerland', 'Cyprus', 'Czechia',
       'Germany', 'Denmark', 'Estonia', 'Greece',
       ...
       'Lithuania', 'Luxembourg', 'Latvia', 'North Macedonia', 'Netherlands',
       'Norway', 'Poland', 'Portugal', 'Sweden', 'Slovakia'],
      dtype='object', name='country', length=3336)

In [ ]:
# import plotly.express as px
# df_grouped = df_mig.groupby(['country', 'sex'])[['2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ']].mean().unstack("country").reset_index()

In [ ]:
# Filter columns with years
year_columns = [col for col in df_mig.columns if col.strip().isdigit()]

# Select only the columns with years along with the relevant identifier columns
df_filtered = df_mig[df_mig['sex'] == 'F'][['citizen', 'agedef', 'age', 'unit', 'sex'] + year_columns]


In [ ]:
# Drop the specified columns
columns_to_drop = ['citizen', 'agedef', 'age', 'unit', 'sex']
df_filtered = df_filtered.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df_filtered

,2021,2020,2019,2018,2017,2016,2015,2014,2013
country,,,,,,,,,
Austria,NaN,NaN,3557.0,3453.0,NaN,4242.0,4203.0,3765.0,3846.0
Belgium,NaN,NaN,1638.0,1504.0,NaN,1171.0,4203.0,1175.0,1640.0
Bulgaria,NaN,NaN,2126.0,1871.0,NaN,1201.0,925.0,869.0,993.0
Switzerland,NaN,NaN,2382.0,2488.0,NaN,2342.0,2264.0,2213.0,2248.0
Cyprus,NaN,NaN,2382.0,2488.0,NaN,2342.0,2264.0,2213.0,24.0
...,...,...,...,...,...,...,...,...,...
Norway,NaN,NaN,71.0,36.0,10.0,35.0,32.0,27.0,37.0
Poland,NaN,NaN,71.0,36.0,10.0,35.0,32.0,27.0,32.0
Portugal,NaN,NaN,71.0,36.0,10.0,35.0,32.0,27.0,0.0


In [ ]:
df_mig_reshaped = df_filtered.stack().reset_index()
df_mig_reshaped.columns = ['Country', 'Year', 'Num_People']
df_grouped = df_mig_reshaped.groupby(['Year','Country'])['Num_People'].mean().unstack("Country")

In [ ]:

# Assuming df_grouped is your grouped DataFrame
# If not, replace it with the actual name of your DataFrame

# Reset the index to make 'Country' and 'Year' regular columns
df_grouped_reset = df_grouped.reset_index()

# Melt the DataFrame to have a single 'Country' column
df_melted = pd.melt(df_grouped_reset, id_vars=['Year'], value_vars=df_grouped_reset.columns[1:],
                    var_name='Country', value_name='Num_People')

# Create the line chart using Plotly Express
fig = px.line(df_melted, x='Year', y='Num_People', color='Country',
              title='Num_People Immigrating Over Time by Country',
              template='plotly_dark')

# fig.update_layout(width=800, height=400)
fig.show()

In [ ]:
# List of columns to keep (exclude "European Union")
columns_to_keep = [col for col in df_filtered.columns if "Euro" not in col]

# Filter columns based on the list
df_filtered = df_filtered[columns_to_keep]

# Calculate mean, median, mode, min, and max for each column
mean_values = df_filtered.mean()
median_values = df_filtered.median()
mode_values = df_filtered.mode().iloc[0]  # Handling multi-modal distributions
min_values = df_filtered.min()
max_values = df_filtered.max()

# Create a new DataFrame with the summary statistics
df_mig_summary = pd.DataFrame({
    'Mean': mean_values,
    'Median': median_values,
    'Mode': mode_values,
    'Min': min_values,
    'Max': max_values
})

# Display or use df_gdp_summary as needed
# df_mig_summary=df_mig.groupby("country")["Mean", "Median", "Mode", "Min", "Max"].sum()
df_mig_summary

,Mean,Median,Mode,Min,Max
2021,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN
2019,151.613309,16.0,0.0,0.0,3557.0
2018,138.637590,15.0,0.0,0.0,3453.0
2017,103.754354,24.0,0.0,0.0,1111.0
2016,130.605216,13.0,0.0,0.0,4242.0
2015,98.217626,7.0,0.0,0.0,4203.0
2014,117.323741,10.0,0.0,0.0,3765.0
2013,155.235612,5.0,0.0,0.0,15953.0


In [ ]:
# # List of columns to keep (exclude "European Union")
# columns_to_keep = [col for col in df_filtered.columns if "Euro" not in col]

# # Filter columns based on the list
# df_filtered = df_filtered[columns_to_keep]

# # Calculate mean, median, mode, min, and max for each column
# mean_values = df_filtered.groupby("country")[['2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ']].mean()
# median_values = df_filtered.groupby("country")[['2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ']].median()
# mode_values = df_filtered.mode().iloc[0]  # Handling multi-modal distributions
# min_values = df_filtered.groupby("country")[['2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ']].min()
# max_values = df_filtered.groupby("country")[['2021 ', '2020 ', '2019 ', '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ']].max()

# # Concatenate the DataFrames along the columns (axis=1)
# concatenated_values = pd.concat([mean_values, median_values, mode_values, min_values, max_values],
#                                 keys=['Mean', 'Median', 'Mode', 'Min', 'Max'], axis=1)

# # Reset the index to make 'country' a regular column
# concatenated_values_reset = concatenated_values.reset_index()

# # Rename the columns to include the statistics type
# concatenated_values_reset.columns = ['country'] + [f'{col[0]}_{col[1]}' for col in concatenated_values_reset.columns[1:]]

# # Display the concatenated DataFrame
# print(concatenated_values_reset)

In [ ]:
# Find the index label where the minimum value in the "Min" column is located
min_row_index = df_mig_summary["Min"].idxmin()

# Extract the entire row based on the index label
min_row = df_mig_summary.loc[min_row_index]

min_row

Mean       151.613309
Median      16.000000
Mode         0.000000
Min          0.000000
Max       3557.000000
Name: 2019 , dtype: float64

In [ ]:
# Find the index label where the maximum value in the "Max" column is located
max_row_index = df_gdp_summary["Max"].idxmax()

# Extract the entire row based on the index label
max_row = df_gdp_summary.loc[max_row_index]

# Display the result
print(max_row)

Mean      5.045670e+08
Median    4.994288e+08
Mode      3.774104e+08
Min       3.749926e+08
Max       6.513644e+08
Name: Germany, dtype: float64


# Goverment Deficits

In [ ]:
url = 'https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&file=data%2Fgov_10dd_edpt1.tsv.gz'

In [ ]:
# Download the TSV file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the response and load it as a DataFrame
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as f:
        df_def = pd.read_csv(f, sep='\t')

    # Now you have the data in a Pandas DataFrame (df_pop)
    # You can work with the DataFrame as needed

else:
    print("Failed to download the file")

In [ ]:
df_def.columns

Index(['unit,sector,na_item,geo\time', '2022 ', '2021 ', '2020 ', '2019 ',
       '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ', '2012 ', '2011 ',
       '2010 ', '2009 ', '2008 ', '2007 ', '2006 ', '2005 ', '2004 ', '2003 ',
       '2002 ', '2001 ', '2000 ', '1999 ', '1998 ', '1997 ', '1996 ', '1995 '],
      dtype='object')

In [ ]:
df_def.index

RangeIndex(start=0, stop=2103, step=1)

In [ ]:
df_def.head()

,"unit,sector,na_item,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,"MIO_EUR,S1,B1GQ,AT",447217.6,405241.4,380888.5,397147.2,385274.1,369361.9,357608.0,344269.2,333146.1,...,242348.3,231862.5,226735.2,220525.1,213606.5,203850.6,195011.9,187853.6,186968.1,184351.3
1,"MIO_EUR,S1,B1GQ,BE",554044.3,507929.6,460747.7,478676.2,460050.8,445050.1,430085.3,416701.4,403003.3,...,296819.7,281200.2,273255.9,264334.9,256376.4,242307.6,231015.7,223032.7,219965.3,220251.5
2,"MIO_EUR,S1,B1GQ,BG",85800.7,71060.1,61607.7,61530.8,56199.8,52501.8,48752.1,45797.8,43024.7,...,21093.2,18798.3,17478.3,15905.3,14440.0,12804.4,13437.6,10064.7,9829.8,14512.8
3,"MIO_EUR,S1,B1GQ,CY",27777.0,24927.6,22086.6,23177.9,21674.0,20312.4,19013.8,17944.2,17482.8,...,13938.0,12871.3,12083.0,11602.9,10804.6,9839.7,9152.6,8414.3,7890.1,7596.0
4,"MIO_EUR,S1,B1GQ,CZ",276229.4,238249.5,215805.4,225613.5,210970.5,194132.9,177438.5,169558.2,157821.3,...,96554.1,88659.5,87358.2,75705.2,67032.5,61082.9,59824.8,54857.3,53088.1,46008.4


In [ ]:
import re
import numpy as np
# Function to clean and convert a column
def clean_and_convert_column(column):
    def clean_and_convert(x):
        try:
            return float(re.sub(r'[^0-9.]', '', str(x)))
        except ValueError:
            return np.nan
    return column.apply(clean_and_convert)

# Apply the function to all columns (excluding the index columns)
for column in df_def.columns:
    if column not in ['unit,sector,na_item,geo\\time']:
        df_def[column] = clean_and_convert_column(df_def[column])

df_def.fillna(method='ffill', inplace=True)

df_def = df_def[~(df_def.iloc[:, 1:] < 100).all(axis=1)]

In [ ]:
# Split the 'column_to_split' into separate columns
# Use double backslashes ('\\') in 'geo\\time' to escape the backslash
# Split the 'citizen,agedef,age,unit,sex,geo\time' column
df_def[['unit','sector','na_item','country']] = df_def['unit,sector,na_item,geo\\time'].str.split(',', expand=True)

# Now, use .loc to assign the values to the desired columns
df_def.loc[:, 'sector'] = df_def['sector'].astype(str)
df_def.loc[:, 'na_item'] = df_def['na_item'].astype(str)
df_def.loc[:, 'unit'] = df_def['unit'].astype(str)
df_def.loc[:, 'country'] = df_def['country'].astype(str)

<ipython-input-150-1f496b2c96c0>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-150-1f496b2c96c0>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-150-1f496b2c96c0>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-150-1f4

In [ ]:
year_columns = [col for col in df_def.columns if col.strip().isdigit()]

# Select only the columns with years along with the relevant identifier columns
df_filtered = df_def[df_def['unit'] == 'MIO_EUR'][['unit', 'sector', 'na_item', 'country'] + year_columns]

In [ ]:
# Map geo\\time to geocode_mapping
df_def.loc[:, "country"] = df_def['country'].map(geocode_mapping)

# Drop the original column if needed
# df_gdp.drop(columns=['unit,na_item,geo\\time', 'unit', 'na_item', 'geo\\time' ], inplace=True)

# Drop rows with NaN index names
df_def.dropna(subset=['country'], inplace=True)

df_def.set_index(["country"], inplace=True)

<ipython-input-152-4e535e4cb799>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-152-4e535e4cb799>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Filter columns with years
year_columns = [col for col in df_def.columns if col.strip().isdigit()]

In [ ]:
columns_to_drop = ['unit', 'sector', 'na_item', 'unit,sector,na_item,geo\time	']
df_filtered = df_filtered.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df_filtered

,country,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,AT,447217.6,405241.4,380888.5,397147.2,385274.1,369361.9,357608.0,344269.2,333146.1,...,242348.3,231862.5,226735.2,220525.1,213606.5,203850.6,195011.9,187853.6,186968.1,184351.3
1,BE,554044.3,507929.6,460747.7,478676.2,460050.8,445050.1,430085.3,416701.4,403003.3,...,296819.7,281200.2,273255.9,264334.9,256376.4,242307.6,231015.7,223032.7,219965.3,220251.5
2,BG,85800.7,71060.1,61607.7,61530.8,56199.8,52501.8,48752.1,45797.8,43024.7,...,21093.2,18798.3,17478.3,15905.3,14440.0,12804.4,13437.6,10064.7,9829.8,14512.8
3,CY,27777.0,24927.6,22086.6,23177.9,21674.0,20312.4,19013.8,17944.2,17482.8,...,13938.0,12871.3,12083.0,11602.9,10804.6,9839.7,9152.6,8414.3,7890.1,7596.0
4,CZ,276229.4,238249.5,215805.4,225613.5,210970.5,194132.9,177438.5,169558.2,157821.3,...,96554.1,88659.5,87358.2,75705.2,67032.5,61082.9,59824.8,54857.3,53088.1,46008.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,PT,1.6,0.4,2.5,0.3,1.5,1.0,1.1,1.8,1.1,...,169.5,198.3,25.8,206.4,51.2,76.6,6.0,4.5,0.0,76.0
697,RO,1684.6,1500.1,1525.0,1075.7,960.4,1132.6,1038.9,794.4,775.4,...,660.9,633.0,784.3,988.2,729.2,952.7,1350.6,1949.2,3331.5,5101.1
698,SE,2454.7,2710.4,2133.2,1823.5,960.4,1132.6,1038.9,794.4,775.4,...,660.9,633.0,784.3,988.2,729.2,952.7,1350.6,1949.2,3331.5,5101.1
699,SI,0.0,0.0,0.1,0.4,0.6,0.8,1.1,1.3,1.5,...,194.4,190.3,142.5,139.4,108.8,952.7,1350.6,1949.2,3331.5,5101.1


In [ ]:
df_filtered = df_filtered.groupby("country")["2022 ","2021 ","2020 ","2019 ","2018 ",
                                             "2017 ","2016 ","2015 ","2014 ","2013 ",
                                             "2012 ","2011 ","2010 ","2009 ","2008 ",
                                             "2007 ","2006 ","2005 ","2004 ","2003 ",
                                             "2002 ","2001 ","2000 ","1999 ","1998 ",
                                             "1997 ","1996 ","1995 "].mean().reset_index()
df_filtered

<ipython-input-156-e2625bd6de94>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,country,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,AT,9.607254e+04,9.063307e+04,8.602574e+04,7.917247e+04,5.697448e+04,5.691866e+04,5.670957e+04,5.625678e+04,5.445224e+04,...,3.415677e+04,3.265918e+04,3.208360e+04,3.088868e+04,2.952968e+04,28144.644000,26432.212000,25601.992000,26626.020000,26612.364000
1,BE,1.420787e+05,1.341037e+05,1.258622e+05,1.170539e+05,1.137811e+05,1.118343e+05,1.105816e+05,1.073900e+05,1.053660e+05,...,7.178088e+04,7.059632e+04,7.036526e+04,6.963088e+04,6.832078e+04,66906.092000,65283.008000,64581.768000,64903.032000,66576.740000
2,BG,1.161225e+04,9.905955e+03,8.643791e+03,8.037473e+03,4.496842e+04,4.401405e+04,4.293760e+04,4.169925e+04,4.082202e+04,...,2.863265e+04,2.782195e+04,2.791945e+04,2.762018e+04,2.726775e+04,26162.381818,25091.313636,24563.045455,61729.281818,63746.913636
3,CY,7.355482e+03,7.257032e+03,7.208395e+03,6.532364e+03,4.465595e+04,4.330360e+04,4.232445e+04,4.131285e+04,4.055781e+04,...,2.855942e+04,2.764780e+04,2.749970e+04,2.691153e+04,2.650613e+04,25474.950000,24337.722727,23859.136364,23496.177273,23789.704545
4,CZ,4.727667e+04,4.053059e+04,3.462800e+04,3.197148e+04,5.733876e+04,5.591694e+04,5.367781e+04,5.298506e+04,5.140978e+04,...,3.378122e+04,3.258840e+04,3.203380e+04,3.054102e+04,2.886106e+04,27350.817391,26266.221739,25292.534783,25060.930435,25438.065217
5,DE,7.517678e+05,7.205928e+05,6.780024e+05,6.332367e+05,5.604736e+05,5.546318e+05,5.468800e+05,5.385971e+05,5.329704e+05,...,3.810914e+05,3.677201e+05,3.569286e+05,3.471195e+05,3.380940e+05,333373.980000,323992.644000,314146.064000,313339.672000,319465.440000
6,DK,5.528999e+04,5.380197e+04,5.126925e+04,4.718260e+04,1.970183e+05,1.913403e+05,1.844708e+05,1.791878e+05,1.749940e+05,...,1.325159e+05,1.278427e+05,1.280984e+05,1.267494e+05,1.244545e+05,271001.391304,261784.843478,252651.078261,249718.152174,245579.960870
7,EA19,3.146514e+06,3.002905e+06,2.831950e+06,2.649048e+06,1.865312e+06,1.835328e+06,1.803230e+06,1.768926e+06,1.738946e+06,...,1.142119e+06,1.094891e+06,1.052318e+06,1.017600e+06,9.800596e+05,954386.300000,926230.944000,898786.304000,891352.224000,859502.884000
8,EA20,3.159590e+06,3.015193e+06,2.843509e+06,2.659995e+06,1.872186e+06,1.842017e+06,1.809799e+06,1.775500e+06,1.745425e+06,...,1.145311e+06,1.097734e+06,1.054857e+06,1.019963e+06,9.822385e+05,956212.320000,927840.540000,900315.176000,892821.084000,860753.492000
9,EE,4.931714e+03,4.295200e+03,3.901890e+03,3.293043e+03,2.326626e+04,2.287377e+04,2.234122e+04,2.179908e+04,2.142275e+04,...,1.452560e+04,2.561647e+04,2.756724e+04,2.723180e+04,2.611552e+04,25565.409524,23984.990476,23099.661905,23030.761905,22245.504762


In [ ]:
# Melt the DataFrame to have 'Year' and 'Value' columns for plotting
df_melted = pd.melt(df_filtered, id_vars='country', var_name='Year', value_name='Euros in Government Deficits')

# Convert 'Year' column to numeric
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Plot the data using a dot plot with Plotly Express
fig = px.scatter(df_melted, x='Year', y='Euros in Government Deficits', color='country',
                 labels={'Value': 'Euros in Government Deficits', 'Year': 'Year'},
                 title='Euros in Government Deficits Over the Years by Country (Dot Plot)')

fig.show()

In [ ]:
px.line(df_melted, x="Year", y='Euros in Government Deficits', title="Euros in Government Deficits in Countries Over the Years", color="country", markers=True)

# Unemployment

In [ ]:
df_emp = pd.read_csv("https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&file=data%2Fei_lmhr_m.tsv.gz")

In [ ]:
# Download the TSV file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the response and load it as a DataFrame
    with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as f:
        df = pd.read_csv(f, sep='\t')

    # Now you have the data in a Pandas DataFrame (df_pop)
    # You can work with the DataFrame as needed

else:
    print("Failed to download the file")

In [ ]:
df

,"unit,sector,na_item,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,"MIO_EUR,S1,B1GQ,AT",447217.6,405241.4,380888.5,397147.2,385274.1,369361.9,357608.0,344269.2,333146.1,...,242348.3,231862.5,226735.2,220525.1,213606.5,203850.6,195011.9,187853.6,186968.1,184351.3
1,"MIO_EUR,S1,B1GQ,BE",554044.3,507929.6,460747.7,478676.2,460050.8,445050.1,430085.3,416701.4,403003.3,...,296819.7,281200.2,273255.9,264334.9,256376.4,242307.6,231015.7,223032.7,219965.3,220251.5
2,"MIO_EUR,S1,B1GQ,BG",85800.7,71060.1,61607.7,61530.8,56199.8,52501.8,48752.1,45797.8,43024.7,...,21093.2,18798.3,17478.3,15905.3,14440.0,12804.4,13437.6,10064.7,9829.8,14512.8
3,"MIO_EUR,S1,B1GQ,CY",27777.0,24927.6,22086.6,23177.9,21674.0,20312.4,19013.8,17944.2,17482.8,...,13938.0,12871.3,12083.0,11602.9,10804.6,9839.7,9152.6,8414.3,7890.1,7596.0
4,"MIO_EUR,S1,B1GQ,CZ",276229.4,238249.5,215805.4,225613.5,210970.5,194132.9,177438.5,169558.2,157821.3,...,96554.1,88659.5,87358.2,75705.2,67032.5,61082.9,59824.8,54857.3,53088.1,46008.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,"PC_GDP,S1314,GD,PT",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.0,0.2,0.0,0.1,0.0,0.0,0.0,0.1
2099,"PC_GDP,S1314,GD,RO",0.6,0.6,0.7,0.5,0.5,0.6,0.6,0.5,0.5,...,1.1,1.4,1.8,2.3,2.2,3.2,4.7,6.8,15.2,22.7
2100,"PC_GDP,S1314,GD,SE",0.5,0.5,0.4,0.4,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
2101,"PC_GDP,S1314,GD,SI",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7,0.7,0.6,0.6,0.5,:,:,:,:,:


In [ ]:
import re
import numpy as np
# Function to clean and convert a column
def clean_and_convert_column(column):
    def clean_and_convert(x):
        try:
            return float(re.sub(r'[^0-9.]', '', str(x)))
        except ValueError:
            return np.nan
    return column.apply(clean_and_convert)

# Apply the function to all columns (excluding the index columns)
for column in df.columns:
    if column not in ['unit,s_adj,indic,geo\time']:
        df[column] = clean_and_convert_column(df[column])

df.fillna(method='ffill', inplace=True)

df = df[~(df.iloc[:, 1:] < 100).all(axis=1)]

In [ ]:
df.columns

Index(['unit,sector,na_item,geo\time', '2022 ', '2021 ', '2020 ', '2019 ',
       '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ', '2012 ', '2011 ',
       '2010 ', '2009 ', '2008 ', '2007 ', '2006 ', '2005 ', '2004 ', '2003 ',
       '2002 ', '2001 ', '2000 ', '1999 ', '1998 ', '1997 ', '1996 ', '1995 '],
      dtype='object')

In [ ]:
df[["unit","s_adj","indic","country"]] = df['unit,sector,na_item,geo\\time'].str.split(',', expand=True)

# Now, use .loc to assign the values to the desired columns
df.loc[:, 'unit'] = df['unit'].astype(str)
df.loc[:, 's_adj'] = df['s_adj'].astype(str)
df.loc[:, 'indic'] = df['indic'].astype(str)
df.loc[:, 'country'] = df['country'].astype(str)

AttributeError: ignored